# 创建driver

In [290]:
from selenium import webdriver
from selenium.webdriver.common.by import By
url = 'http://live.titan007.com/index2in1.aspx?id=3'
driver = webdriver.Chrome(r'C:\Users\StZhmoka22\Desktop\chromedriver.exe')
driver.get(url)

<ipython-input-290-2bcaa7dd5014>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\StZhmoka22\Desktop\chromedriver.exe')


# 输入关键字并跳转到新网页

In [90]:
'''text1 = driver.find_element(By.XPATH,'//input[@type="text"][@id="search-input"][@name="q"][@placeholder="请输入检索关键字"]')
text1.send_keys('上海新增')
text1.send_keys('\ue007')
driver.switch_to.window(driver.window_handles[1])'''

# 网页基本设置

## 赛事筛选

In [291]:
driver.find_element(By.XPATH,'//*[@id="button3"]').click() # ‘赛事’按钮点击
driver.find_element(By.XPATH,'//*[@id="Layer2"]/div[3]/input[2]').click() # '全选'按钮点击
driver.find_element(By.XPATH,'//*[@id="Layer2"]/div[3]/input[3]').click() # '反选'按钮点击
favorite_match = ['X西杯','L罗甲'] # 以澳威甲，马来超为例，创建感兴趣的赛事列表,注意一定要在单个元素前添加首字母拼音大写

# 热门赛事筛选
for match_name in favorite_match:
    class_n = match_name[0]
    match_name_CH = match_name[1:]
    match_text = driver.find_element(By.XPATH,'//*[@id="classHot"]').text
    match_list = match_text.split('\n')
    for index,mn in enumerate(match_list):
        if match_name_CH in mn and match_name_CH == mn[:len(match_name_CH)]:
            driver.find_element(By.XPATH,'//*[@id="classHot"]'+'/label['+str(index+1)+']/span').click()
            
# 拼音开头赛事筛选
for match_name in favorite_match:
    class_n = match_name[0]
    match_name_CH = match_name[1:]
    match_text = driver.find_element(By.XPATH,'//*[@id="class'+ class_n+ '"]').text
    match_list = match_text.split('\n')
    for index,mn in enumerate(match_list):
        if match_name_CH in mn and match_name_CH == mn[:len(match_name_CH)]:
            driver.find_element(By.XPATH,'//*[@id="class'+ class_n+ '"]'+'/label['+str(index+1)+']/span').click()

driver.find_element(By.XPATH,'//*[@id="Layer2"]/div[3]/input[4]').click() # 点击确定
'''driver.find_element(By.XPATH,'//*[@id="result-info-inner"]/div[1]/ul[1]/li[1]').click()
driver.find_element(By.XPATH,'//*[@id="main-body"]/div[6]/div[3]/div[1]/ul/li[5]/span').click()
driver.find_element(By.XPATH,'//*[@id="dateFilterRange"]').send_keys('2022-04-02 至 2022-05-21')
driver.find_element(By.XPATH,'//*[@id="customTime"]').click()'''

'driver.find_element(By.XPATH,\'//*[@id="result-info-inner"]/div[1]/ul[1]/li[1]\').click()\ndriver.find_element(By.XPATH,\'//*[@id="main-body"]/div[6]/div[3]/div[1]/ul/li[5]/span\').click()\ndriver.find_element(By.XPATH,\'//*[@id="dateFilterRange"]\').send_keys(\'2022-04-02 至 2022-05-21\')\ndriver.find_element(By.XPATH,\'//*[@id="customTime"]\').click()'

## 其他设置

In [41]:
# 您这边的需求和网页默认设置相同，故不做处理，后续有变动的话，再做迭代修改

## 功能选择

In [292]:
driver.find_element(By.XPATH,'//*[@id="tools"]/ul/li[6]').click()# 点击‘功能选择’
driver.find_element(By.XPATH,'//*[@id="onlyTopShowWin"]').click()# 点选‘仅提示置顶赛事弹框’
driver.find_element(By.XPATH,'//*[@id="rank"]').click()# 点选‘排名’
driver.find_element(By.XPATH,'//*[@id="Layer3"]/div[1]/span[2]/a').click()# 点击‘x’关闭功能选择弹窗 

# 赛事页面

## 定义文本处理为df的函数

In [288]:
def text_function(text,match_info_text):
    import pandas as pd
    new_list = []
    data = text.split('\n')
    keys = data[0].split(' ')
    keys[-2] = '变化时间（日期）'
    keys.insert(-1,'变化时间（时分）')
    for item in data[1:]:
        if '封' in item:
            item = item.replace(' 封 ',' 封 封 封 ')
            new_list.append(item.split(' '))
        else:
            new_list.append(item.split(' '))
    for index in range(len(new_list)):
        if len(new_list[index]) != 8:
            new_list[index] = ['未开始','未开始'] + new_list[index]
    df = pd.DataFrame(new_list,columns = keys)
    save_root = r'D:\\BaiduNetdiskDownload\\简历\\足球数据情报\\'+match_info_text+keys[2]+'vs'+keys[4]+'.csv'
    df.to_csv(save_root)

## 具体步骤

In [293]:
analysis_list = driver.find_elements(By.XPATH,'//td[@class="icons2"]/a[text() = "析"]') # 获取所有'析'元素对应的XPATH

# 进入所选赛事数据分析页面，设置并取数
for elements in analysis_list:
    try:
        elements.click() # 点击筛选出的赛事对应的‘析’元素
        driver.switch_to.window(driver.window_handles[1]) # 移动操作窗口至新窗口
        # 赛事分析页面设定
        driver.find_element(By.XPATH,'//*[@id="right_float"]/a[1]').click() # 点击‘定制’
        driver.find_element(By.XPATH,'//*[@id="mySet"]/div/input[1]').click() # 点击‘全选’
        driver.find_element(By.XPATH,'//*[@id="mySet"]/div/input[2]').click() # 点击‘确定’
        # 盘口设定
        driver.find_element(By.XPATH,'//*[@id="hType_hn"]/option[1]').click() # 主队盘口设置为‘初’盘
        driver.find_element(By.XPATH,'//*[@id="hType_an"]/option[1]').click() # 客队盘口设置为‘初’盘
        # 过滤无效数据
        match_class_element = driver.find_elements(By.XPATH,'//*[@id="hn_l"]/label')# 获取主队近10场比赛所包含的比赛属性
        match_class_list = [index for index,item in enumerate(match_class_element) if item.text[-1] == '杯' or item.text[-2:] == '友谊'] # 获取以‘杯’或者‘友谊’结尾的index
        input_match_class = driver.find_elements(By.NAME,'hn_l')# 获取对应勾选元素对应的XPATH
        for index in match_class_list:
            input_match_class[index].click()# 点掉以'杯'或‘友谊’结尾的选项
        match_class_element = driver.find_elements(By.XPATH,'//*[@id="an_l"]/label')# 获取客队近10场比赛所包含的比赛属性
        match_class_list = [index for index,item in enumerate(match_class_element) if item.text[-1] == '杯' or item.text[-2:] == '友谊'] # 获取以‘杯’或者‘友谊’结尾的index
        input_match_class = driver.find_elements(By.NAME,'an_l')# 获取对应勾选元素对应的XPATH
        for index in match_class_list:
            input_match_class[index].click()# 点掉以'杯'或‘友谊’结尾的选项
        # 历史数据抓取
        pankou_home = driver.find_elements(By.XPATH,'//*[@id="vsNew"]/tbody/tr/td/table[1]/tbody/*//td[8]/a')# 获取主队近10场所有开盘比赛的页面列表
        pankou_away = driver.find_elements(By.XPATH,'//*[@id="vsNew"]/tbody/tr/td/table[2]/tbody/*//td[8]/a')# 获取客队近10场所有开盘比赛的页面列表
        pankou = pankou_home+pankou_away
        for web in pankou:
            web.click()
            driver.switch_to.window(driver.window_handles[2])# 移动至盘口页面
            match_info = driver.find_element(By.XPATH,'/html/body/div[3]/div[1]/div[3]/div[1]').text.split(' ')[:2]# 获取赛事名称及比赛时间信息
            match_info_text = ''.join(match_info)
            web_company = driver.find_elements(By.XPATH,'//*[@id="odds"]/tbody/tr')# 获取所有开盘庄家具体信息
            # 定位Crow公司所在位置
            for index,wc in enumerate(web_company):
                if wc.text[:4] == 'Crow':
                    driver.find_element(By.XPATH,'//*[@id="odds"]/tbody/tr['+str(index+1)+']/td[12]/a[1]').click()# 点击‘Crow’对应的‘详’
                    break
            driver.switch_to.window(driver.window_handles[3])# 移动至Crow水位变化详情页
            text = driver.find_element(By.XPATH,'//*[@id="odds2"]/table/tbody').text# 获取Crow水位变化详情页数据
            text_function(text,match_info_text)# 存储数据为csv
            driver.close()# 关闭driver.window_handles[3]页面
            driver.switch_to.window(driver.window_handles[2])# 移动至盘口页面
            driver.close()# 关闭driver.window_handles[2]页面
            driver.switch_to.window(driver.window_handles[1])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])# 赛事筛选完成页面
    except:
        pass

In [267]:
            driver.close()# 关闭driver.window_handles[3]页面
            driver.switch_to.window(driver.window_handles[2])# 移动至盘口页面
            driver.close()# 关闭driver.window_handles[2]页面
            driver.switch_to.window(driver.window_handles[1])# 移动至'析'页面
            driver.close()
            driver.switch_to.window(driver.window_handles[0])# 赛事筛选完成页面

30

In [279]:
driver.find_elements(By.XPATH,'//*[@id="vsNew"]/tbody/tr/td/table[1]/tbody/*//td[8]/a')

[<selenium.webdriver.remote.webelement.WebElement (session="cde50caf804a0b3e41b5f53379ea9290", element="fcd1dd6d-6d04-48e7-9b76-f3eba85f680f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cde50caf804a0b3e41b5f53379ea9290", element="e9efcc09-ccdd-4cbb-bb01-ace054480319")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cde50caf804a0b3e41b5f53379ea9290", element="339f5902-1e3b-4d6f-a08b-5e78349faf57")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cde50caf804a0b3e41b5f53379ea9290", element="b9e7caf0-f99d-4fb9-a5b1-cdb6f341420d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cde50caf804a0b3e41b5f53379ea9290", element="dd29408a-a802-4191-89d5-48b3491d8f41")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cde50caf804a0b3e41b5f53379ea9290", element="390d5887-0e23-450c-84b2-71f463c1ec06")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cde50caf804a0b3e41b5f53379ea9290", element="8e6aacd7-f9a6-40b5-8119-f0

In [176]:

web_company = driver.find_elements(By.XPATH,'//*[@id="odds"]/tbody/tr')# 获取所有开盘庄家具体信息
# 定位Crow公司所在位置
for index,wc in enumerate(web_company):
    if wc.text[:4] == 'Crow':
        driver.find_element(By.XPATH,'//*[@id="odds"]/tbody/tr['+str(index+1)+']/td[12]/a[1]').click()# 点击‘Crow’对应的‘详’
        break

In [191]:
text = driver.find_element(By.XPATH,'//*[@id="odds2"]/table/tbody').text
text.split('\n')

['时间 比分 哈柯亚东悉尼市 盘口 坎特贝利班克斯敦 变化时间 状态',
 '90 2-0 封 2-4 18:20 滚',
 '88 2-0 0.99 平手 0.85 2-4 18:18 滚',
 '88 2-0 0.95 平手 0.89 2-4 18:18 滚',
 '87 2-0 0.99 平手 0.85 2-4 18:17 滚',
 '86 2-0 0.97 平手 0.87 2-4 18:16 滚',
 '83 2-0 0.92 平手 0.92 2-4 18:14 滚',
 '83 2-0 0.97 平手 0.87 2-4 18:13 滚',
 '82 2-0 0.92 平手 0.92 2-4 18:12 滚',
 '82 2-0 0.88 平手 0.96 2-4 18:12 滚',
 '80 2-0 0.90 平手 0.94 2-4 18:10 滚',
 '79 2-0 0.91 平手 0.93 2-4 18:10 滚',
 '79 2-0 0.90 平手 0.94 2-4 18:10 滚',
 '79 2-0 0.91 平手 0.93 2-4 18:09 滚',
 '78 1-0 封 2-4 18:08 滚',
 '78 1-0 0.90 平手 0.94 2-4 18:08 滚',
 '76 0-0 封 2-4 18:07 滚',
 '76 0-0 0.73 平手 1.12 2-4 18:07 滚',
 '76 0-0 0.78 平手 1.06 2-4 18:06 滚',
 '72 0-0 0.83 平手 1.01 2-4 18:02 滚',
 '69 0-0 0.84 平手 1.00 2-4 17:59 滚',
 '69 0-0 0.79 平手 1.05 2-4 17:59 滚',
 '69 0-0 0.84 平手 1.00 2-4 17:59 滚',
 '68 0-0 0.79 平手 1.05 2-4 17:58 滚',
 '65 0-0 0.76 平手 1.08 2-4 17:55 滚',
 '65 0-0 0.72 平手 1.13 2-4 17:54 滚',
 '64 0-0 0.77 平手 1.07 2-4 17:54 滚',
 '63 0-0 0.72 平手 1.13 2-4 17:53 滚',
 '62 0-0 0.67 平手 1.20 2